In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.pardir)))

In [2]:
import shutil

# Set up a clean test output directory
TEST_DIR = "test"
if os.path.exists(TEST_DIR):
    shutil.rmtree(TEST_DIR)
os.makedirs(TEST_DIR, exist_ok=True)

In [3]:
import pandas as pd
import numpy as np

# Define the scenario directory
SCENARIO_PATH = os.path.join(TEST_DIR, "scenario1")
os.makedirs(SCENARIO_PATH, exist_ok=True)

# Define the results directory
RESULTS_DIR = "results"
os.makedirs(RESULTS_DIR, exist_ok=True)

# Helper function to create and display a CSV file from a DataFrame
def create_csv(df, filename):
    path = os.path.join(SCENARIO_PATH, filename)
    df.to_csv(path, index=False)

# Use otoole to initialize the scenario structure
!otoole setup csv test/scenario1/ --overwrite

In [4]:
# User Settings
HOURS_PER_YEAR = 8760

# TIME
years = np.linspace(2026, 2027, num=1, dtype=int)
seasons = ['ALLSEASONS']
daytypes = ['ALLDAYS']
timebrackets = ['ALLTIMES']

# TOPOLOGY
regions = ['REGION1']

# DEMAND
demand = np.tile([100], len(years)) # MWh per year

# SUPPLY
technologies = ['GAS_CCGT', 'GAS_TURBINE']
residual_capacity = [0, 0] # MW
max_capacity = [1000/HOURS_PER_YEAR, 1000/HOURS_PER_YEAR] # MW
min_capacity = [0, 0] # MW

# PERFORMANCE
operating_life = [30, 25] # years
efficiency = [0.5, 0.4] # fraction of input energy converted to output energy
capacity_factors = [0.9, 0.8] # fraction of max capacity
availability = [1.0, 1.0] # fraction of year

# ECONOMICS
discount_rate = 0.05 
capital_costs = [500, 400] # $ / MW ($ per unit of extended capacity)
fixed_costs = [0, 0] # $ / MW / year ($ per unit of installed capacity per year)
variable_costs = [2, 5] # $ / MWh ($ per unit of activity)

# STORAGE (NOT IMPLEMENTED IN THIS EXAMPLE)

# EMISSIONS (NOT IMPLEMENTED IN THIS EXAMPLE)

# TARGETS (NOT IMPLEMENTED IN THIS EXAMPLE)

In [5]:
# YEAR
year_df = pd.DataFrame({'VALUE': years})
create_csv(year_df, 'YEAR.csv')

# REGION
region_df = pd.DataFrame({'VALUE': regions})
create_csv(region_df, 'REGION.csv')

# TECHNOLOGY
tech_df = pd.DataFrame({'VALUE': technologies})
create_csv(tech_df, 'TECHNOLOGY.csv')

# FUEL
fuel_df = pd.DataFrame({'VALUE': ['ELEC', 'GAS']})
create_csv(fuel_df, 'FUEL.csv')

# MODE_OF_OPERATION
mode_df = pd.DataFrame({'VALUE': [1]})
create_csv(mode_df, 'MODE_OF_OPERATION.csv')

# SEASON
season_df = pd.DataFrame({'VALUE': seasons})
create_csv(season_df, 'SEASON.csv')

# DAYTYPE
daytype_df = pd.DataFrame({'VALUE': daytypes})
create_csv(daytype_df, 'DAYTYPE.csv')

# DAILYTIMEBRACKET
dailytimebracket_df = pd.DataFrame({'VALUE': timebrackets})
create_csv(dailytimebracket_df, 'DAILYTIMEBRACKET.csv')

# TIMESLICE
timeslices = [f"{s}_{d}_{t}" for s in seasons for d in daytypes for t in timebrackets]
timeslice_df = pd.DataFrame({'VALUE': timeslices})
create_csv(timeslice_df, 'TIMESLICE.csv')

In [6]:
# SpecifiedAnnualDemand
demand_df = pd.DataFrame({
    'REGION': regions * len(years),
    'FUEL': ['ELEC'] * len(years),
    'YEAR': years,
    'VALUE': demand # MWh
})
create_csv(demand_df, 'SpecifiedAnnualDemand.csv')

# SpecifiedDemandProfile
demand_profile_df = pd.DataFrame({
    'REGION': regions * len(years)*len(timeslices),
    'FUEL': ['ELEC'] * len(years)*len(timeslices),
    'TIMESLICE': np.array([timeslices] * len(years)).flatten(),
    'YEAR': np.array([years] * len(timeslices)).flatten(),
    'VALUE': [1.0/len(timeslices)] * len(years)
    # Proportion (%) of annual demand
    # CURRENTLY EVENLY DISTRIBUTED ACROSS TIMESLICES
})
create_csv(demand_profile_df, 'SpecifiedDemandProfile.csv')

# CapacityFactor
capacity_factor_df = pd.DataFrame({
    'REGION': regions * len(technologies)*len(years),
    'TECHNOLOGY': np.repeat(technologies, len(years)*len(timeslices)),
    'TIMESLICE': np.array([timeslices] * len(technologies)*len(years)).flatten(),
    'YEAR': np.array([years] * len(technologies)*len(timeslices)).flatten(),
    'VALUE': np.repeat(capacity_factors, len(years)*len(timeslices))
    # Capacity factors (%) for each technology across all timeslices and years
    # CURRENTLY ONE VALUE PER TECHNOLOGY
})
create_csv(capacity_factor_df, 'CapacityFactor.csv')

# AvailabilityFactor
availability_factor_df = pd.DataFrame({
    'REGION': regions * len(technologies)*len(years),
    'TECHNOLOGY': np.repeat(technologies, len(years)),
    'YEAR': np.array([years] * len(technologies)).flatten(),
    'VALUE': np.repeat(availability, len(years))
    # Availability factors (%) for each technology and year
    # CURRENTLY ONE VALUE PER TECHNOLOGY
})
create_csv(availability_factor_df, 'AvailabilityFactor.csv')

# CapitalCost
capital_cost_df = pd.DataFrame({
    'REGION': regions * len(technologies)*len(years),
    'TECHNOLOGY': np.repeat(technologies, len(years)),
    'YEAR': np.array([years] * len(technologies)).flatten(),
    'VALUE': np.repeat(capital_costs, len(years))
    # Capital cost [$ per MWh]
    # CURRENTLY ONE VALUE PER TECHNOLOGY
})
create_csv(capital_cost_df, 'CapitalCost.csv')

# FixedCost
fixed_cost_df = pd.DataFrame({
    'REGION': regions * len(technologies)*len(years),
    'TECHNOLOGY': np.repeat(technologies, len(years)),
    'YEAR': np.array([years] * len(technologies)).flatten(),
    'VALUE': np.repeat(fixed_costs, len(years))
    # Fixed cost [$ per MWh per year]
    # CURRENTLY ONE VALUE PER TECHNOLOGY
})
create_csv(fixed_cost_df, 'FixedCost.csv')

# CapacityToActivityUnit
capacity_to_activity_df = pd.DataFrame({
    'REGION': regions * len(technologies),
    'TECHNOLOGY': technologies,
    'VALUE': [HOURS_PER_YEAR] * len(technologies)
    # Energy that would be produced when one unit of capacity is fully used in one year
    # (MWh / capacity unit) * 8760 hours per year
    # CURRENTLY ONE VALUE PER TECHNOLOGY
    # (this explicitly sets 1 capacity unit = 1 MW, 1 activity unit = 1 hour at full capacity)
})
create_csv(capacity_to_activity_df, 'CapacityToActivityUnit.csv')

# VariableCost
variable_cost_df = pd.DataFrame({
    'REGION': regions * len(technologies)*len(years),
    'TECHNOLOGY': np.repeat(technologies, len(years)),
    'MODE_OF_OPERATION': [1] * len(technologies)*len(years),
    'YEAR': np.array([years] * len(technologies)).flatten(),
    'VALUE': np.repeat(variable_costs, len(years))
    # Variable cost [$ per activity unit]
    # (since 1 activity unit = 1 hour at full capacity, this is $ / hour)
    # CURRENTLY ONE VALUE PER TECHNOLOGY
})
create_csv(variable_cost_df, 'VariableCost.csv')

# InputActivityRatio
input_ratio_df = pd.DataFrame({
    'REGION': regions * len(technologies)*len(years),
    'TECHNOLOGY': np.repeat(technologies, len(years)),
    'FUEL': np.repeat(['GAS'], len(technologies)*len(years)),
    'MODE_OF_OPERATION': [1] * len(technologies)*len(years),
    'YEAR': np.array([years] * len(technologies)).flatten(),
    'VALUE': np.repeat([1.0]/np.array(efficiency), len(years))
    # MWh fuel / MWh electricity
    # CURRENTLY ONE VALUE PER TECHNOLOGY
})
create_csv(input_ratio_df, 'InputActivityRatio.csv')

# OutputActivityRatio
output_ratio_df = pd.DataFrame({
    'REGION': regions * len(technologies)*len(years),
    'TECHNOLOGY': np.repeat(technologies, len(years)),
    'FUEL': np.repeat(['ELEC'], len(technologies)*len(years)),
    'MODE_OF_OPERATION': [1] * len(technologies)*len(years),
    'YEAR': np.array([years] * len(technologies)).flatten(),
    'VALUE': np.repeat([1.0]*len(technologies), len(years))
    # MWh electricity / MWh electricity
    # CURRENTLY ONE VALUE PER TECHNOLOGY
})
create_csv(output_ratio_df, 'OutputActivityRatio.csv')

In [7]:
# Conversionls
conv_ls_df = pd.DataFrame({
    'TIMESLICE': timeslices,
    'SEASON': np.array([seasons] * len(timeslices)).flatten(),
    'VALUE': [1 if ts.startswith(s) else 0 for ts in timeslices for s in seasons]
})
create_csv(conv_ls_df, 'Conversionls.csv')

# Conversionld
conv_ld_df = pd.DataFrame({
    'TIMESLICE': timeslices,
    'DAYTYPE': np.array([daytypes] * len(timeslices)).flatten(),
    'VALUE': [1 if f"_{daytypes[0]}_" in ts else 0 for ts in timeslices for s in daytypes]
})
create_csv(conv_ld_df, 'Conversionld.csv')

# Conversionlh
conv_lh_df = pd.DataFrame({
    'TIMESLICE': timeslices,
    'DAILYTIMEBRACKET': np.array([timebrackets] * len(timeslices)).flatten(),
    'VALUE': [1 if ts.endswith(h) else 0 for ts in timeslices for h in timebrackets]
})
create_csv(conv_lh_df, 'Conversionlh.csv')

# DaysInDayType
days_in_day_type_df = pd.DataFrame({
    'SEASON': np.array([seasons] * len(years)*len(daytypes)).flatten(),
    'DAYTYPE': np.array([daytypes] * len(years)*len(seasons)).flatten(),
    'YEAR': np.array([years] * len(daytypes)*len(seasons)).flatten(),
    'VALUE': np.array([365/len(daytypes)/len(seasons)] * len(years)*len(seasons)*len(daytypes)).flatten()
    # LEAP YEARS?
    # np.array([365] * len(years)) + (years % 4 == 0)
})
create_csv(days_in_day_type_df, 'DaysInDayType.csv')

# DaySplit
day_split_df = pd.DataFrame({
    'DAILYTIMEBRACKET': np.array([timebrackets] * len(years)).flatten(),
    'YEAR': years,
    'VALUE': np.array([1/(len(timebrackets) * 365)] * len(years)).flatten()
    # Length of one timebracket in one specific day as a fraction of the year
})
create_csv(day_split_df, 'DaySplit.csv')

# YearSplit
year_split_df = pd.DataFrame({
    'TIMESLICE': np.array([timeslices] * len(years)).flatten(),
    'YEAR': years,
    'VALUE': np.array([1/len(timeslices)] * len(years)).flatten()
    # Duration of a modelled timeslice as a fraction of the year
})
create_csv(year_split_df, 'YearSplit.csv')


In [8]:
# OperationalLife
op_life_df = pd.DataFrame({
    'REGION': regions * len(technologies),
    'TECHNOLOGY': technologies,
    'VALUE': operating_life
})
create_csv(op_life_df, 'OperationalLife.csv')

# ResidualCapacity
residual_cap_df = pd.DataFrame({
    'REGION': regions * len(technologies)*len(years),
    'TECHNOLOGY': np.repeat(technologies, len(years)),
    'YEAR': np.array([years] * len(technologies)).flatten(),
    'VALUE': np.repeat(residual_capacity, len(years))
})
create_csv(residual_cap_df, 'ResidualCapacity.csv')

# TotalAnnualMaxCapacity
total_max_cap_df = pd.DataFrame({
    'REGION': regions * len(technologies) * len(years),
    'TECHNOLOGY': np.repeat(technologies, len(years)),
    'YEAR': np.array([years] * len(technologies)).flatten(),
    'VALUE': np.repeat(max_capacity, len(years))
})
create_csv(total_max_cap_df, 'TotalAnnualMaxCapacity.csv')

# TotalAnnualMinCapacity
total_min_cap_df = pd.DataFrame({
    'REGION': regions * len(technologies)*len(years),
    'TECHNOLOGY': np.repeat(technologies, len(years)),
    'YEAR': np.array([years] * len(technologies)).flatten(),
    'VALUE': np.repeat(min_capacity, len(years))
})
create_csv(total_min_cap_df, 'TotalAnnualMinCapacity.csv')

# Discount Rate
discount = pd.DataFrame({
    'REGION': regions,
    'VALUE': [discount_rate] * len(regions)
    # CURRENTLY ONE VALUE PER REGION
})
create_csv(discount, 'DiscountRate.csv')

In [9]:
!otoole convert csv datafile test/scenario1 test/scenario1.txt ../docs/OSeMOSYS_config.yaml

In [10]:
!glpsol -m ../docs/OSeMOSYS.txt -d test/scenario1.txt --wglp test/scenario1.glp --write test/scenario1.sol

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 -m ../docs/OSeMOSYS.txt -d test/scenario1.txt --wglp test/scenario1.glp --write
 test/scenario1.sol
Reading model section from ../docs/OSeMOSYS.txt...
1425 lines were read
Reading data section from test/scenario1.txt...
165 lines were read
Checking Max and Min capcity-investment bounds for r in REGION, t in TECHNOLOGY, y in YEAR 
Checking (line 175)...
Checking Annual activity limits for r in REGION, t in TECHNOLOGY, y in YEAR 
Checking (line 180)...
Checking Residual and TotalAnnualMax Capacity for r in REGION, t in TECHNOLOGY, y in YEAR 
Checking (line 185)...
Checking Residual, Total annual maxcap and mincap investments for  all Region, Tech and Year 
Checking (line 190)...
Checking Annual production by technology bounds for r in REGION, t in TECHNOLOGY, y in YEAR 
Checking (line 195)...
Checking TimeSlices/YearSplits for y in YEAR 
Checking (line 200)...
Checking (line 201)...
Checking Model period activity

In [11]:
!otoole results glpk csv test/scenario1.sol results datafile test/scenario1.txt ../docs/OSeMOSYS_config.yaml --glpk_model test/scenario1.glp

In [12]:
# Load and Display OSeMOSYS Results
# Print the results
print("\n--- OSeMOSYS Optimization Results ---")
osemosys_objective = pd.read_csv(os.path.join(RESULTS_DIR, 'TotalDiscountedCost.csv'))
print("\nObjective:", osemosys_objective)
print("--- OSeMOSYS: Optimal Capacities ---")
osemosys_total_capacity = pd.read_csv(os.path.join(RESULTS_DIR, 'TotalCapacityAnnual.csv'))
print("\nOptimal Capacities (p_nom_opt):\n", osemosys_total_capacity)
osemosys_total_production = pd.read_csv(os.path.join(RESULTS_DIR, 'TotalTechnologyAnnualActivity.csv'))
print("\nTotal Production:\n", osemosys_total_production)


--- OSeMOSYS Optimization Results ---

Objective:     REGION  YEAR       VALUE
0  REGION1  2026  195.572923
--- OSeMOSYS: Optimal Capacities ---

Optimal Capacities (p_nom_opt):
     REGION   TECHNOLOGY  YEAR     VALUE
0  REGION1     GAS_CCGT  2026  0.012684
1  REGION1  GAS_TURBINE  2026  0.000000

Total Production:
     REGION   TECHNOLOGY  YEAR  VALUE
0  REGION1     GAS_CCGT  2026  100.0
1  REGION1  GAS_TURBINE  2026    0.0


In [13]:
# Create the equivalent PyPSA network
import pypsa
from pypsa.common import annuity

# Initialize the network
n = pypsa.Network()

# Add carriers
n.add("Carrier", "GAS")
n.add("Carrier", "AC") # ELEC

# Add the buses
n.add("Bus", region_df['VALUE'], carrier="AC")

# Set investment period
periods = year_df['VALUE'].tolist()
n.set_investment_periods(periods)

# Set investment period weightings
investment_period_weightings = pd.DataFrame(index=years)
investment_period_weightings['years'] = 1  # Each period represents 1 year
investment_period_weightings['objective'] = [
    1 / ((1 + discount_rate) ** (y - years[0])) for y in years
]
n.investment_period_weightings = investment_period_weightings

# Set snapshots
timesteps = timeslice_df['VALUE'].tolist()
snapshots = pd.MultiIndex.from_product([periods, timesteps], names=['period', 'timestep'])
n.set_snapshots(snapshots)

# Set snapshot weightings
year_split_df['period'] = year_split_df['YEAR']
year_split_df['timestep'] = year_split_df['TIMESLICE']
hours_per_snapshot = year_split_df.set_index(['period', 'timestep'])['VALUE'] * HOURS_PER_YEAR
hours_per_snapshot = hours_per_snapshot.reindex(snapshots)
n.snapshot_weightings['objective'] = hours_per_snapshot
n.snapshot_weightings['generators'] = hours_per_snapshot

demand_pypsa = pd.merge(
    demand_df,
    demand_profile_df,
    on=['REGION', 'FUEL', 'YEAR'],
    suffixes=('_annual', '_profile')
)
# Energy per snapshot = Annual Demand × Demand Profile
demand_pypsa['energy_MWh'] = demand_pypsa['VALUE_annual'] * demand_pypsa['VALUE_profile']
demand_pypsa['period'] = demand_pypsa['YEAR']
demand_pypsa['timestep'] = demand_pypsa['TIMESLICE']

# Power = Energy / Hours
energy_series = demand_pypsa.set_index(['period', 'timestep'])['energy_MWh']
power = energy_series.divide(hours_per_snapshot).reindex(snapshots)

n.add("Load",
      "demand",
      bus="REGION1",
      p_set=power)  # MW

# Add generators
p_max_pu = pd.DataFrame(index=snapshots) # Set time-dependent availability (CapacityFactor)
for idx, tech in enumerate(technologies):
    total_annual_cost = capital_costs[idx] * annuity(operating_life[idx], discount_rate) + fixed_costs[idx] # $ / MW of extended capacity / year
    n.add("Generator",
          tech,
          bus="REGION1",
          carrier="GAS",
          p_nom_max=max_capacity[idx], # MW
          p_nom_min=min_capacity[idx], # MW
          p_nom_extendable=True,
          capital_cost=capital_costs[idx], 
          marginal_cost=variable_costs[idx],
          efficiency = efficiency[idx],
          lifetime=operating_life[idx],
          build_year=years[0] # new capacity can be built from year 0 onwards
          )

    p_max_pu[tech] = capacity_factors[idx]
n.generators_t.p_max_pu = p_max_pu

# Print network components to verify
print("--- PyPSA Network Components ---")
# print("\nBuses:\n", n.buses)
print("\nGenerators:\n", n.generators[['p_nom', 'p_nom_extendable', 'marginal_cost', 'efficiency']])
# print("\nLoads:\n", n.loads)
print("\nTime-varying load:\n", n.loads_t.p_set)
print("\nTime-varying generator availability:\n", n.generators_t.p_max_pu)
print("\nSnapshot Weightings:\n", n.snapshot_weightings)

INFO:pypsa.network.index:Repeating time-series for each investment period and converting snapshots to a pandas.MultiIndex.


--- PyPSA Network Components ---

Generators:
              p_nom  p_nom_extendable  marginal_cost  efficiency
name                                                           
GAS_CCGT       0.0              True            2.0         0.5
GAS_TURBINE    0.0              True            5.0         0.4

Time-varying load:
 name                                  demand
period timestep                             
2026   ALLSEASONS_ALLDAYS_ALLTIMES  0.011416

Time-varying generator availability:
                                     GAS_CCGT  GAS_TURBINE
period timestep                                          
2026   ALLSEASONS_ALLDAYS_ALLTIMES       0.9          0.8

Snapshot Weightings:
                                     objective  stores  generators
period timestep                                                  
2026   ALLSEASONS_ALLDAYS_ALLTIMES     8760.0     1.0      8760.0


In [14]:
# Run the PyPSA optimization
n.optimize(solver_name='glpk', compute_infeasibilities=True, multi_investment_periods=True)

INFO:linopy.model: Solve problem using Glpk solver
INFO:linopy.io: Writing time: 0.01s
INFO:linopy.solvers:GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --lp /tmp/linopy-problem-3iv6xp04.lp --output /tmp/linopy-solve-wd_y7x4i.sol
Reading problem data from '/tmp/linopy-problem-3iv6xp04.lp'...
9 rows, 4 columns, 12 non-zeros
55 lines were read
GLPK Simplex Optimizer 5.0
9 rows, 4 columns, 12 non-zeros
Preprocessing...
3 rows, 4 columns, 6 non-zeros
Scaling...
 A: min|aij| =  8.000e-01  max|aij| =  1.000e+00  ratio =  1.250e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 3
      0: obj =   5.000000000e+02 inf =   1.142e-02 (1)
      2: obj =   5.057077626e+02 inf =   0.000e+00 (0)
*     4: obj =   2.063419584e+02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (40424 bytes)
Writing basic solution to '/tmp/linopy-solve-wd_y7x4i.sol'...

INFO:linopy.constants: Optimizatio

('ok', 'optimal')

In [15]:
# Print the results
print("\n--- PyPSA Optimization Results ---")
print("\nObjective:", n.objective)
print("\nOptimal Capacities (p_nom_opt):\n", n.generators.p_nom_opt)
print("\nTotal Production (p * HOURS_PER_YEAR):\n", n.generators_t.p * HOURS_PER_YEAR)


--- PyPSA Optimization Results ---

Objective: 206.3419584

Optimal Capacities (p_nom_opt):
 name
GAS_CCGT       0.012684
GAS_TURBINE    0.000000
Name: p_nom_opt, dtype: float64

Total Production (p * HOURS_PER_YEAR):
 name                                GAS_CCGT  GAS_TURBINE
period timestep                                          
2026   ALLSEASONS_ALLDAYS_ALLTIMES  99.99978          0.0


OSeMOSYS: NPV with Salvage Value

OSeMOSYS computes Net Present Value of all costs, crediting back the residual value of assets that outlive the model horizon.

Inputs:
- NewCapacity = 0.012684 MW
- CapitalCost = 500 $/MW
- VariableCost = 2 $/MWh
- OperationalLife = 30 years
- DiscountRate = 0.05
- Model Period = 1 year (2026)

1. Capital Investment:
   - CapitalInvestment = 500 × 0.012684 = $6.34
   - DiscountedCapitalInvestment = $6.34 / (1.05)^0 = $6.34

2. Salvage Value (asset life extends beyond model):
   - Since (2026 + 30 - 1) = 2055 > 2026, salvage applies.
   
   - SV = 6.34 × (1 - [(1.05)^1 - 1] / [(1.05)^30 - 1])
      = 6.34 × (1 - 0.05 / 3.322)
      = 6.34 × 0.985
      = $6.24
   
   - DiscountedSalvageValue = 6.24 / (1.05)^1 = $5.94

3. Operating Cost:
   - OperatingCost = 100 MWh × 2 $/MWh = $200
   - DiscountedOperatingCost = 200 / (1.05)^0.5 = $195.18 (mid-year)

4. Total Discounted Cost:
   = DiscountedCapital + DiscountedOperating - DiscountedSalvage
   = 6.34 + 195.18 - 5.94
   = $195.58

PyPSA: Multi-investment Period Cost Accounting

When using `multi_investment_periods=True` with total (non-annualized) capital costs, PyPSA computes costs as follows:

Inputs:
- NewCapacity = 0.012684 MW
- capital_cost = 500 $/MW (total investment, NOT annualized)
- marginal_cost = 2 $/MWh
- Production = 100 MWh
- investment_period_weightings['objective'] = 1.0 (for 2026)

1. Capital Investment:
   - CapitalCost × Capacity × InvestmentPeriodWeight
   = 500 × 0.012684 × 1.0
   = $6.34

2. Operating Cost:
   - MarginalCost × Energy × InvestmentPeriodWeight
   = 2 × 100 × 1.0
   = $200.00

3. Total:
   = $6.34 + $200.00
   = $206.34


Reconciling the Difference
| Component | PyPSA | OSeMOSYS | Difference |
| --------- | ----- | -------- | ---------- |
| Capital Investment | $6.34 | $6.34 | $0.00 |
| Salvage Value Credit | $0.00 | −$5.94 | +$5.94 |
| Operating Cost | $200.00 | $195.18 | +$4.82 |
| Total | $206.34 | $195.58 | +$10.76 |

In [16]:
n.statistics()

,,Optimal Capacity,Installed Capacity,Supply,Withdrawal,Energy Balance,Transmission,Capacity Factor,Curtailment,Capital Expenditure,Operational Expenditure,Revenue,Market Value
,,2026,2026,2026,2026,2026,0,2026,2026,2026,2026,2026,2026
Generator,GAS,0.01268,NaN,99.99978,NaN,99.99978,0.0,0.900631,0.00009,6.34195,199.99956,206.34201,2.063047
Load,-,0.00000,0.0,NaN,100.0,-100.00000,0.0,NaN,0.00000,0.00000,0.00000,-206.34247,NaN


PyPSA: Curtailment

| Metric | Value |
| ------ | ----- |
| Optimal Capacity | 0.012684 MW |
| Input p_max_pu (Capacity Factor limit) | 0.9 |
| Hours | 8760 |
| Maximum Available Generation | 0.012684 × 0.9 × 8760 = 100.008 MWh |
| Actual Supply	| 99.99978 MWh |
| Curtailment | 0.00009 MWh |
| Load | 100.0 MWh |

Curtailment is defined as:
Curtailment = Available Generation − Actual Generation
            = p_nom_opt × p_max_pu × hours - Demand

- Curtailment is an artifact of floating-point arithmetic precision, LP solver convergence tolerance
- Need to find an $\epsilon$ to flag large differences